In [67]:
# import pkgs
import pandas as pd
import glob
import locale
import numpy as np
from Demos.win32cred_demo import target

locale.setlocale(locale.LC_ALL, 'fa_IR.UTF-8')


'fa_IR.UTF-8'

In [68]:
# Functions

def quantile_score(vec, score):
    scorevec = np.zeros(len(vec))
    qu = np.quantile(vec, np.linspace(0, 1, score + 1))
    scorevec[(vec <= qu[1]) & (vec >= qu[0])] = 1
    for i in range(1, score - 1):
        scorevec[(vec <= qu[i + 1]) & (vec > qu[i])] = i + 1
    scorevec[vec > qu[score]] = score
    return scorevec

def generate_dates(years, months, days30, days31):
    dates = []
    for month in months:
        if month in Months:
            dates.append(f"{years}{month}{days31}")
        else:
            dates.append(f"{years}{month}{days30}")
    return dates


In [94]:
# Parameters

start_date_jalali = 14030601
target_date_jalali = 14031130
start_date = date(2024, 8, 22)  # Start train
target_date = date(2025, 2, 18)  # End train

# Years = ["1403"]
# Months = ["{:02d}".format(i) for i in range(1, 13)]  # Generates "01" to "12"
# Days30 = ["{:02d}".format(i) for i in range(1, 31)]  # Generates "01" to "30"
# Days31 = ["{:02d}".format(i) for i in range(1, 32)]  # Generates "01" to "31"
#
# print("Years:", Years)
# print("Months:", Months)
# print("Days30:", Days30)
# print("Days31:", Days31)
#
# main_dates = generate_dates(Years, Months, Days30, Days31)
# # Remove specific dates
# excluded_dates = {14021031, 14021131, 14021231, 14030731, 14030831, 14030931, 14031031, 14031131, 14031231}
# main_dates = [date for date in main_dates if date not in excluded_dates]

In [78]:
# type(start_date_jalali)

int

In [79]:
# Read Data
# Folder containing Parquet files
folder_path = "Data/data/*.parquet"

# List all parquet files
parquet_files = glob.glob(folder_path)

# Read and concatenate all files
df = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)

DimDate = pd.read_csv("Data/data/DimDate.csv")


In [80]:
import pandas as pd
# Preperation
# Convert date column to string

df["date_CHR"] = df["date"].astype(str)

# Create Miladi_Num by extracting and concatenating substrings
df["Miladi_Num"] = df["date_CHR"].str[:4] + df["date_CHR"].str[5:7] + df["date_CHR"].str[8:10]

# Convert Miladi column to string
DimDate["Miladi_CHR"] = DimDate["Miladi"].astype(str)

# Create Miladi_Num in DimDate
DimDate["Miladi_Num"] = DimDate["Miladi_CHR"].str[:4] + DimDate["Miladi_CHR"].str[5:7] + DimDate["Miladi_CHR"].str[8:10]

# Select specific columns
DimDateS = DimDate[["Jalali_1", "Miladi_Num"]]

# Left join on Miladi_Num
df = df.merge(DimDateS, on="Miladi_Num", how="left")

# Create Shamsi_Date and convert to numeric
df["Shamsi_Date_Num"] = (df["Jalali_1"].str[:4] + df["Jalali_1"].str[5:7] + df["Jalali_1"].str[8:10]).astype(int)


In [81]:
# Filter & Select
# Assuming df is already a pandas DataFrame
TrainRFM = df[df["module"] == "Onlineshopping"]

TrainRFM = TrainRFM[TrainRFM["payment_status"] == "payed"]
TrainRFM = TrainRFM[TrainRFM["status"] == "finished"]

print(TrainRFM.columns)
print(TrainRFM.shape)

# select data for modeling
TrainRFM = TrainRFM[['user_id', '_id', 'initial_total', 'Shamsi_Date_Num']]

# print(TrainRFM.head(5))

Index(['_id', 'user_id', 'date', 'client_type', 'status', 'payment_status',
       'type', 'module', 'product_name', 'initial_total', 'product_price',
       'converted_value', 'qty', 'module_unit_price', 'Amount_in_dollars',
       'first_purchased', 'date_CHR', 'Miladi_Num', 'Jalali_1',
       'Shamsi_Date_Num'],
      dtype='object')
(503726, 20)
                         user_id                       _id  initial_total  \
274229  60b0025888157467350fc9fe  62378f72b002627c54389cbc      2200000.0   
274237  60e34a06b31930354d285c3b  623790a314bf061ad531ff18     37537500.0   
274264  60b0012cd7651d2f521ea565  6237960fd55f160171061f0d     11550000.0   
274507  60b00cfe6248355acf0f4cfa  62381428a4a78610e2786852     14148750.0   
274508  60b008608815746735105ecf  6238145b0b4f097aab447dfa      8247250.0   

        Shamsi_Date_Num  
274229         14010101  
274237         14010101  
274264         14010101  
274507         14010101  
274508         14010101  


In [82]:
# make R, F, M
# Group by user_id and calculate min and max dates
TrainRFM_RB = TrainRFM.groupby("user_id", as_index=False).agg(
    minDate=("Shamsi_Date_Num", "min"),
    maxDate=("Shamsi_Date_Num", "max")
)

# TrainRFM_RB.head(5)

,user_id,minDate,maxDate
0,60affeafd7651d2f521e1512,14030909,14030909
1,60affeafd7651d2f521e152c,14010430,14010430
2,60affeb1d7651d2f521e155a,14010631,14010804
3,60affeb1d7651d2f521e1565,14011006,14011006
4,60affeb1d7651d2f521e1584,14010102,14010929


In [83]:
# TrainRFM_RB.dtypes

user_id    object
minDate     int32
maxDate     int32
dtype: object

In [95]:
# !pip install jdatetime
import jdatetime
import datetime
from datetime import date
# from datetime import datetime


def jalali_to_miladi(jalali_date):
    # Extract year, month, day from the integer Jalali date
    year = jalali_date // 10000
    month = (jalali_date % 10000) // 100
    day = jalali_date % 100

    # Convert to Gregorian using jdatetime
    gregorian_date = jdatetime.date(year, month, day).togregorian()

    # Return formatted Gregorian date
    return gregorian_date


TrainRFM_RB['maxDate_Miladi'] = TrainRFM_RB['maxDate'].apply(jalali_to_miladi)


# TrainRFM_RB['maxDate_Miladi'] = TrainRFM_RB['maxDate_Miladi'].apply(
#     lambda x: datetime.strptime(x, '%Y-%m-%d').date()
# )

TrainRFM_RB = TrainRFM_RB[(TrainRFM_RB['maxDate'] >= start_date_jalali) & (TrainRFM_RB['maxDate'] <= target_date_jalali)]

TrainRFM_RB['R'] = TrainRFM_RB['maxDate_Miladi'].apply(lambda d: (target_date - d).days)


# print(TrainRFM_RB.head(5))


                     user_id   minDate   maxDate maxDate_Miladi    R
0   60affeafd7651d2f521e1512  14030909  14030909     2024-11-29   81
5   60affeb1d7651d2f521e158e  14030220  14030906     2024-11-26   84
6   60affeb1d7651d2f521e1591  14030924  14030924     2024-12-14   66
11  60affeb3d7651d2f521e16b9  14020113  14031120     2025-02-08   10
12  60affeb3d7651d2f521e16d9  14010117  14030711     2024-10-02  139
